![](https://images.pexels.com/photos/1194713/pexels-photo-1194713.jpeg)

# Using SSDP to Find Rokus on a Network

Kevin Walchko, Phd

8 Apr 2021

---

The Simple Service Discovery Protocol (SSDP) is a network protocol based on the Internet Protocol Suite for advertisement and discovery of network services and presence information. It accomplishes this without assistance of server-based configuration mechanisms, such as DHCP or DNS, and without special static configuration of a network host. SSDP is the basis of the discovery protocol of Universal Plug and Play (UPnP) and is intended for use in residential or small office environments. It was formally described in an IETF Internet draft by Microsoft and Hewlett-Packard in 1999. Although the IETF proposal has since expired (April, 2000), SSDP was incorporated into the UPnP protocol stack, and a description of the final implementation is included in UPnP standards documents.

In [1]:
import socket
import http
from io import StringIO, BytesIO
from colorama import Fore

In [122]:
# Modified version of Dan Krause (2014) example for Python3
# Licensed under the Apache License, Version 2.0 (the "License");
class SSDPResponse(object):
    class _FakeSocket(BytesIO):
        def makefile(self, *args, **kw):
            return self
    def __init__(self, response):
        print(response)
        r = http.client.HTTPResponse(self._FakeSocket(response))
        r.begin()
        self.resp = r
        h = r.getheaders()
        # print(h)
        for (k,v) in h:
            # fix keys
            k=k.replace('-','_')
            k=k.replace('.','_')
            setattr(self, k, v)

def discover(service, timeout=1, retries=2, mx=1):
    # Roku's use a defined multicast address and port 
    group = ("239.255.255.250", 1900)
    
    # standard message to find them
    message = "\r\n".join([
        'M-SEARCH * HTTP/1.1',
        f'HOST: {group[0]}:{group[1]}',
        'MAN: "ssdp:discover"',
        f'ST: {service}',
        f'MX: {mx}','',''])
    
    socket.setdefaulttimeout(timeout)
    responses = {}
    for _ in range(retries):
        # create a UDP multicast socket and repeat in this for loop
        sock = socket.socket(
            socket.AF_INET, 
            socket.SOCK_DGRAM, 
            socket.IPPROTO_UDP)
        # enable reuse on the socket, so others can use it too
        sock.setsockopt(
            socket.SOL_SOCKET, 
            socket.SO_REUSEADDR, 
            1)
        # set the time-to-live for the UDP packet
        sock.setsockopt(
            socket.IPPROTO_IP, 
            socket.IP_MULTICAST_TTL, 
            2)
        # broadcast it out to the network
        sock.sendto(
            message.encode("utf-8"), 
            group)
        
        # keep trying to read until the socket timesout
        while True:
            try:
                response = SSDPResponse(sock.recv(1024))
                responses[response.USN] = response
            except socket.timeout:
                break
    return responses

In [129]:
resp = discover("roku:ecp")
print(resp)

b'HTTP/1.1 200 OK\r\nCache-Control: max-age=3600\r\nST: roku:ecp\r\nUSN: uuid:roku:ecp:YG00AE380198\r\nExt: \r\nServer: Roku/10.5.0 UPnP/1.0 Roku/10.5.0\r\nLOCATION: http://10.0.1.101:8060/\r\ndevice-group.roku.com: 429ED23BAD44850BC822\r\n\r\n'
b'HTTP/1.1 200 OK\r\nCache-Control: max-age=3600\r\nST: roku:ecp\r\nUSN: uuid:roku:ecp:YG00CM052475\r\nExt: \r\nServer: Roku/10.5.0 UPnP/1.0 Roku/10.5.0\r\nLOCATION: http://10.0.1.24:8060/\r\ndevice-group.roku.com: 429ED23BAD44850BC822\r\n\r\n'
b'HTTP/1.1 200 OK\r\nCache-Control: max-age=3600\r\nST: roku:ecp\r\nUSN: uuid:roku:ecp:YG00CM052475\r\nExt: \r\nServer: Roku/10.5.0 UPnP/1.0 Roku/10.5.0\r\nLOCATION: http://10.0.1.24:8060/\r\ndevice-group.roku.com: 429ED23BAD44850BC822\r\n\r\n'
b'HTTP/1.1 200 OK\r\nCache-Control: max-age=3600\r\nST: roku:ecp\r\nUSN: uuid:roku:ecp:YG00AE380198\r\nExt: \r\nServer: Roku/10.5.0 UPnP/1.0 Roku/10.5.0\r\nLOCATION: http://10.0.1.101:8060/\r\ndevice-group.roku.com: 429ED23BAD44850BC822\r\n\r\n'
{'uuid:roku:ecp:YG

In [124]:
for k,v in resp.items():
    print(f"Roku found on: {k}")
    # print(f"  {dir(v)}")
    print(f"\t{v.Cache_Control}")
    # print(f"\t{v.Ext}")
    print(f"\t{v.LOCATION}")
    print(f"\t{v.ST}")
    print(f"\t{v.Server}")
    print(f"\t{v.USN}")
    print(f"\t{v.device_group_roku_com}")
    # print(f"\t{v.resp}")

Roku found on: uuid:roku:ecp:YG00CM052475
	max-age=3600
	http://10.0.1.24:8060/
	roku:ecp
	Roku/10.5.0 UPnP/1.0 Roku/10.5.0
	uuid:roku:ecp:YG00CM052475
	429ED23BAD44850BC822
Roku found on: uuid:roku:ecp:YG00AE380198
	max-age=3600
	http://10.0.1.101:8060/
	roku:ecp
	Roku/10.5.0 UPnP/1.0 Roku/10.5.0
	uuid:roku:ecp:YG00AE380198
	429ED23BAD44850BC822
